In [1]:
import json 
import pprint
import jieba
import re
from gensim.corpora import WikiCorpus
from collections import Counter

D:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def cut_text_list(textlist):
    import jieba
    jieba.load_userdict(r'D:\WordLibrary\JiebaUse\jieba_dict.txt.big')
    with open(r"D:\WordLibrary\JiebaUse\stopwords.txt",encoding='utf8') as sw:
        stopwords=sw.read().split("\n")
    textcutlist=[]
    for text in textlist:
        cutwords=jieba.cut(text)
        cuttext=" ".join([word for word in cutwords if word not in stopwords and '\u4e00' <= word <= '\u9fff'])
        textcutlist.append(cuttext)
    return textcutlist

def word_count(textlist,n=300):
    textcutlist=cut_text_list(textlist)
    from collections import Counter
    wordCountlist = [Counter(i.split(" ")).most_common(n) for i in textcutlist]
    return wordCountlist

def TF_IDF(textlist,n=300):
    textcutlist=cut_text_list(textlist)
    from collections import Counter
    wordCountlist = [Counter(i.split(" ")).most_common(n) for i in textcutlist]
    TfList = [[((j[0], j[1] / sum(dict(i).values()))) for j in i] for i in wordCountlist]
    wordlist = []
    for i in wordCountlist:
        for j in i:
            wordlist.append(j[0])
    wordlist = set(wordlist)
    Ntext = len(textlist)
    wordappear = {}
    for i in wordlist:
        n = 0
        for j in textcutlist:
            if i in j:
                n += 1
        wordappear[i] = n
    import math
    IDFlist = {i: math.log(Ntext / wordappear[i], 10) for i in wordappear}
    TF_IDFlist = [Counter({j[0]: j[1] * IDFlist[j[0]] for j in i}) for i in TfList]
    return TF_IDFlist


In [3]:
# with open(r'D:\Data\JsonData\TainanFood\TainanUnique.json') as f:
#     TainanDatas=json.load(f)
with open(r'D:\Data\JsonData\TainanFood\AnalyzeTable.json') as f:
    TainanAnalyzeDatas=json.load(f)

In [4]:
TainanAnalyzeDatas[1].keys()

dict_keys(['name', 'style', 'Ncomment', 'wordCount', 'TF_IDF', 'contentcut'])

In [5]:
# for i in TainanAnalyzeDatas:
#     try:
#         del i['TF_IDF'][i['name']]
#         del i['TF_IDF']['']
#     except:
#         pass
# from collections import Counter
# for i in TainanAnalyzeDatas:   
#     i["tag"]="、".join([j[0] for j in Counter(i["TF_IDF"]).most_common(3)])
# for i in TainanAnalyzeDatas:
#     if i["tag"]=="":
#         TainanAnalyzeDatas.remove(i)

In [6]:
for i in TainanAnalyzeDatas:
    try:
        del i['TF_IDF'][i['name']]
        del i['TF_IDF']['']
    except:
        pass
from collections import Counter
for i in TainanAnalyzeDatas:   
    i["tag"]=i["style"]
# for i in TainanAnalyzeDatas:
#     if i["tag"]=="":
#         TainanAnalyzeDatas.remove(i)

In [40]:
TainanAnalyzeDatas=[i for i in TainanAnalyzeDatas if i['tag']!='其他小吃' and i['tag']!='複合式']
Counter([i["tag"] for i in TainanAnalyzeDatas]).most_common(50)

[('咖啡專賣', 282),
 ('飲料專賣店', 248),
 ('其它中式料理', 202),
 ('義式料理', 170),
 ('其他鍋類', 152),
 ('台菜餐廳', 132),
 ('美式料理', 132),
 ('剉冰、豆花', 131),
 ('其他日式料理', 126),
 ('綜合日式料理', 125),
 ('炭烤串燒', 115),
 ('熱炒、快炒', 103),
 ('休閒零食', 101),
 ('西式早餐', 98),
 ('其他美食', 83),
 ('冰淇淋', 81),
 ('海鮮餐廳', 81),
 ('其他異國料理', 79),
 ('中式', 74),
 ('麻辣火鍋', 66),
 ('粵菜 港式飲茶', 61),
 ('生魚片、壽司專賣', 60),
 ('漢堡、炸雞', 59),
 ('日式麵食專賣', 57),
 ('韓式料理', 56),
 ('泰式料理', 54),
 ('果汁', 51),
 ('甜品、甜湯', 46),
 ('其它主題餐廳', 46),
 ('涮涮鍋', 45),
 ('鹹酥雞、炸雞排', 43),
 ('酒吧 Lounge Bar', 42),
 ('居酒屋', 41),
 ('茶專賣', 40),
 ('山產野菜餐廳', 39),
 ('中式早餐', 37),
 ('麵攤、麵店', 35),
 ('法式料理', 34),
 ('自助餐 便當', 34),
 ('其他類型早餐', 31),
 ('鐵板燒', 31),
 ('越南料理', 28),
 ('日式豬排專賣', 25),
 ('四川菜', 21),
 ('食補藥膳餐廳', 21),
 ('懷舊主題', 19),
 ('臭豆腐', 19),
 ('披薩', 19),
 ('中式糕餅', 18),
 ('滷味', 15)]

In [41]:
import pandas

In [42]:
Tainantable=pandas.DataFrame(TainanAnalyzeDatas)
Tainantable

,Ncomment,TF_IDF,contentcut,name,style,tag,wordCount
0,20,"{'豆花': 0.10713037550139132, '同記安平豆花': 0.033821...",簡單 傳統 安平 豆花 純白 豆花 特別 挑逗 胃口 安平 兼具 台南 傳統 現代 一個 區...,同記安平豆花(安平總店),剉冰、豆花,剉冰、豆花,"{'豆花': 206, '吃': 87, '安平': 63, '台南': 41, '珍珠':..."
1,20,"{'食下': 0.04852748992166938, '蹄膀': 0.0307641703...",天氣 冷 吃 個鍋 內容 比較 一篇 介紹 無 制式 餐點 本篇 介紹 花雕 鍋 有別於 之...,食下有約 想法廚房,其他異國料理,其他異國料理,"{'吃': 162, '約': 157, '食下': 149, '料理': 111, '花雕..."
2,20,"{'土壤': 0.01937361722482767, '典藏': 0.0131905478...",飛 饗 義 法典 藏 料理 旗艦 總店 週日 上班 日 下班 美食 好好 犒賞 同事 跑 飛...,飛饗創意西式料理,義式料理,義式料理,"{'吃': 160, '麵': 121, '飯': 112, '沙拉': 110, '燉':..."
3,20,"{'水果': 0.02981049752400656, '莉莉': 0.0283074655...",台南美食 特別 傳統 美食 歷久不衰 今天 拜訪 開 一甲子 老牌 水果店 莉莉水果店 中午...,莉莉水果店,剉冰、豆花,剉冰、豆花,"{'水果': 117, '台南': 83, '吃': 75, '冰': 66, '莉莉水果店..."
4,20,"{'煎餅': 0.10441180752447335, '連得堂': 0.042770751...",台南 火車站 前站 出發 走路 分鐘 即可 抵達 當天 只 家人 小妹 依稀 記憶 親切 老...,連得堂餅舖,休閒零食,休閒零食,"{'煎餅': 200, '雞蛋': 68, '台南': 63, '味': 56, '吃': ..."
5,20,"{'哈密瓜': 0.05801891744189857, '瓜瓜': 0.027751450...",吃 完 霜淇淋 再 對面 泰成水果店 傳統 水果 行 店 使用 人 超多 併 桌翻 桌率 很...,泰成水果店,果汁,果汁,"{'吃': 131, '哈密瓜': 112, '水果': 101, '冰': 99, '台南..."
6,20,"{'騷烤': 0.04313402840394703, '油條': 0.0165410389...",之前 住 日木 本來 騷烤 家 吃 晚餐 當天 組 部落 客 約 約 喬 喬 昨天 終於 阿...,騷烤家(前鋒總店),炭烤串燒,炭烤串燒,"{'烤': 124, '吃': 115, '騷烤': 94, '家': 91, '好': 6..."
7,20,"{'芒果': 0.043025357578805705, '草莓': 0.036412558...",一位 難求 台南 人氣 名店 冰鄉 瞧瞧 草莓 牛乳 冰 上面 滿滿的 草莓 碩大 鮮豔 欲...,冰鄉,剉冰、豆花,剉冰、豆花,"{'冰': 168, '芒果': 153, '吃': 144, '草莓': 137, '冰鄉..."
8,20,"{'銅鍋': 0.044760053254128934, '舒芙蕾': 0.02825549...",位於 德安 百貨 小 銅鍋 提供 多樣 義式料理 排餐 招牌 舒芙蕾 飲品 甜食 環境 寬敞...,小銅鍋義式餐廳(德安誠品店),義式料理,義式料理,"{'小': 249, '吃': 237, '銅鍋': 210, '麵': 152, '舒芙蕾..."
9,20,"{'薏仁': 0.1078265031169122, '抹茶': 0.03500514760...",介紹 完 國華 街上 小卷 米粉 布萊恩 紅茶 之後 遺漏掉 近幾年 爆紅 菜市 之花 薏仁...,大菜市純薏仁,甜品、甜湯,甜品、甜湯,"{'薏仁': 254, '抹茶': 126, '台南': 96, '吃': 90, '紅豆'..."


In [43]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from pprint import pprint

In [44]:
#文字轉向量矩陣
count_vect=CountVectorizer()
X_train_counts=count_vect.fit_transform(Tainantable.contentcut)

In [45]:
tfidf_transformer=TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Rocchio

In [46]:

from sklearn.neighbors.nearest_centroid import NearestCentroid
clf = NearestCentroid().fit(X_train_tfidf,Tainantable.tag)

In [47]:
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)
predicted =clf.predict(x_new_tfidf)

In [48]:
for doc,category in zip(docs_new,predicted):
    print("%r=>%s"%(doc,category))

'食記 壹等賞 桃園 龍潭 看板 批 踢踢 實業 坊 批 踢踢 實業 坊 看板 聯絡 資訊 返回 看板 分享 作者 看板 標題 食記 壹等賞 桃園 龍潭 時間 本文 轉錄 看板 作者 看板 標題 食記 壹等賞 桃園 龍潭 時間 圖文 版 今天 提早到 新竹 火車站 附近 髮 瑟 剪 頭髮 整個 人 變得 清爽 免不了 念 幾句 想 以後 頭髮 剪 好 今天 比較 晚 下班 六點 多才 離開 公司 交通 正 處於 尖峰 時刻 光復路 繞 一圈 還了 漫畫 之後 決定 晚上 吃 桃園 龍潭 壹等賞 新竹 文化 街上 紅葉 告訴 目前 肚子 不餓 想 說 現在 開到 壹等賞 時間 應該 夠長 肚子 應該 餓 決定 上次 曾經 吃過 一次 壹等賞 享用 晚餐 壹等賞 位於 龍潭 渴望 園區 附近 上次 來過 一次 不知 道路 走 知道 高速公路 累得 睡著 開 十分鐘 下 關西 交 流道 辦法 睡 一下 一下 交流 道 吵醒 請 幫 帶路 今 天 再 走過 一次 不太 認得 路 晚上 天色 黑 另一方 面是 山路 白天 去過 難記 得住 路要 走 下面 名片 背後 地圖 有興 趣 參考 一下 地址 桃園縣 龍潭 鄉 高原 村 中原 路 二段 巷 弄 號 電話 營業時間 每週 公休 今天 一到 停車場 滿滿的 車子 嚇到 已經 七點 多了還 一堆 人 一看 發現 應該 公司 聚餐 很多人 穿著 同樣 制服 外面 抽 飯後 煙 空間 很大 晚到 位置 坐 壹等賞 一個 茶園 旁邊 一片 很大 茶 田 裡面 餐廳 賣 茶葉 東西 就連 洗手間 洗手 乳 茶葉 香味 服務 人員 一間 廂房 裡面 已經 坐著 滿滿的 客人 一到 看到 客人 本來 已經 心理準備 廚房 上菜 比較慢 一點 一點 完菜 不到 三分鐘 菜 一道 一道 送上來 真 佩服 廚房 裡面 大廚 燒菜 功力 上菜 食物 好吃 熱茶 免費 餐桌上 一壺 大壺 熱茶 免費 提供 飲用 醃菜 頭 免費 喜歡 吃 只 吃 兩塊 剩下 吃 光光 用來 搭配 稀飯 一起 吃 最好 稀飯 點 好吃 茶油 拌 飯 珍香蜜 茶 鵝 小 茶鵝 很好吃 有點 貴 外面 賣 鵝肉 不太 應該 有用 茶葉 燻 很香 養生茶 油拌 飯 點了 兩碗 四碗 普通 碗 份量 要點 茶油 拌 飯 吃 哪有 吃白飯 道理 桂花 蜜 排骨 小 糖醋排骨 

In [49]:
predicted

array(['台菜餐廳', '美式料理', '義式料理', '甜點專賣', '其他鍋類', '', '臭豆腐', '', '義式料理',
       '美式料理', '其它中式料理', '泰式料理', '台菜餐廳'], dtype=object)

# bayes

In [50]:

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,Tainantable.tag)
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)
predicted =clf.predict(x_new_tfidf)
predicted

array(['咖啡專賣', '咖啡專賣', '咖啡專賣', '咖啡專賣', '咖啡專賣', '咖啡專賣', '其它中式料理', '咖啡專賣',
       '咖啡專賣', '咖啡專賣', '咖啡專賣', '咖啡專賣', '咖啡專賣'],
      dtype='<U13')

# kmeans

In [51]:

from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(15).fit(X_train_tfidf,Tainantable.tag)
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)
predicted =clf.predict(x_new_tfidf)
predicted

array(['咖啡專賣', '休閒零食', '果汁', '咖啡專賣', '海鮮餐廳', '海鮮餐廳', '其它中式料理', '其他日式料理',
       '飲料專賣店', '其他鍋類', '冰淇淋', '其他鍋類', '中式'], dtype=object)

# svm

In [52]:
from sklearn import svm
clf = svm.SVC(kernel="linear").fit(X_train_tfidf,Tainantable.tag)
with open(r'D:\Data\testData\foodptt.json') as f:
    testdatas=json.load(f)
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)
predicted =clf.predict(x_new_tfidf)
predicted

array(['其它中式料理', '咖啡專賣', '義式料理', '咖啡專賣', '其他鍋類', '飲料專賣店', '臭豆腐', '飲料專賣店',
       '咖啡專賣', '西式早餐', '其它中式料理', '泰式料理', '其它中式料理'], dtype=object)

In [53]:
Counter(Tainantable.tag)

Counter({'': 1,
         '上海菜(江浙菜)': 8,
         '下午茶': 2,
         '中式': 74,
         '中式早餐': 37,
         '中式糕餅': 18,
         '休閒零食': 101,
         '俄羅斯料理': 1,
         '其他亞洲料理': 14,
         '其他日式料理': 126,
         '其他異國料理': 79,
         '其他美食': 83,
         '其他速食': 12,
         '其他鍋類': 152,
         '其他類型buffet自助餐': 2,
         '其他類型早餐': 31,
         '其它中式料理': 202,
         '其它主題餐廳': 46,
         '冰淇淋': 81,
         '刈包': 4,
         '剉冰、豆花': 131,
         '動漫主題餐廳': 1,
         '北京菜': 3,
         '印度料理': 2,
         '台菜餐廳': 132,
         '咖啡專賣': 282,
         '啤酒屋': 4,
         '四川菜': 21,
         '墨西哥料理': 5,
         '壽喜燒': 6,
         '客家菜': 5,
         '寵物餐廳': 1,
         '小吃': 10,
         '居酒屋': 41,
         '山產野菜餐廳': 39,
         '巧克力專賣': 2,
         '希臘料理': 1,
         '德式料理': 6,
         '情境主題餐廳': 6,
         '懷石料理': 1,
         '懷舊主題': 19,
         '披薩': 19,
         '文字燒、大阪燒': 2,
         '日式': 1,
         '日式燒肉': 14,
         '日式豬排專賣': 25,
         '日式麵食專賣': 57,
       

In [ ]:
# rocchio 
# array(['台菜餐廳', '美式料理', '義式料理', '西點烘焙、麵包', '其他鍋類', '', '臭豆腐', '', '義式料理',
#        '美式料理', '麵食點心', '泰式料理', '台菜餐廳'], dtype=object)
# bayes
# array(['麵食點心', '咖啡專賣', '咖啡專賣', '西點烘焙、麵包', '麵食點心', '麵食點心', '麵食點心',
#        '西點烘焙、麵包', '咖啡專賣', '麵食點心', '麵食點心', '麵食點心', '麵食點心'],
#       dtype='<U13')
# kmeans
# array(['其他鍋類', '炭烤串燒', '休閒零食', '麵食點心', '麵食點心', '西式早餐', '咖啡專賣', '其它中式料理',
#        '海鮮餐廳', '中式', '飲料專賣店', '台菜餐廳', '海鮮餐廳'], dtype=object)
# svm
# array(['其它中式料理', '咖啡專賣', '義式料理', '西點烘焙、麵包', '麵食點心', '麵食點心', '臭豆腐', '麵食點心',
#        '咖啡專賣', '麵食點心', '麵食點心', '泰式料理', '其它中式料理'], dtype=object)

In [62]:
testdatas=['咖哩 好難吃 但 這裡 的 重點 是 鍋貼 跟 水餃 還有 乾拌麵','天阿 今天 的 阿忠 麵線 好好吃','今天 的 叉燒 真 好吃','這個 世界 好 可怕 除了 薯條 跟 炸雞  以外',"今天 去 知名度 吃 牛排 他 的 牛排 真得 很 好吃 而且 還 附贈 玉米 濃湯 沙拉 紅茶 冰淇淋 無現 吃到 飽 現在 還有 豬排 魚排 麵包 可以吃"]
docs_new=cut_text_list(testdatas)
X_new_counts= count_vect.transform(docs_new)
x_new_tfidf=tfidf_transformer.transform(X_new_counts)
predicted =clf.predict(x_new_tfidf)
predicted

array(['飲料專賣店', '飲料專賣店', '日式麵食專賣', '漢堡、炸雞', '美式料理'], dtype=object)